In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# GPU나 MPS 같은 하드웨어 가속기를 사용할 수 있는지 확인
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
# 신경망 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
# 모델 구조 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
# 무작위 입력 데이터로 모델 테스트
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


In [ ]:
# 확인용 코드: 사용 가능한 CUDA 아키텍처 출력
import torch; torch.cuda.get_arch_list()

['sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90', 'sm_100', 'sm_120']

In [ ]:
# 3개의 미니 Batch 가지고 오기
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [ ]:
# flatten 계층 사용해보기
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [ ]:
# linear 계층 사용해보기
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [ ]:
# ReLU 활성화 함수 사용해보기
print(f"Before ReLU : {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU : {hidden1}")

Before ReLU : tensor([[-0.1756,  0.0896, -0.0317, -0.1471, -0.3847, -0.1080, -0.2202, -0.1798,
         -0.3148,  0.0387,  0.2980, -0.1488, -0.7490,  0.0761,  0.3098,  0.0652,
         -0.3429, -0.1003, -0.0580,  0.4092],
        [-0.1019, -0.3935,  0.0308, -0.0745, -0.2862, -0.0479, -0.1696,  0.3762,
         -0.1636, -0.0557, -0.0539, -0.2019, -0.6882,  0.1843,  0.3023, -0.3334,
         -0.1771, -0.1059,  0.0497,  0.4190],
        [-0.0585, -0.1627,  0.1001, -0.2004, -0.1731,  0.1498,  0.0636, -0.1887,
         -0.2309, -0.2197,  0.1777, -0.1109, -0.6620,  0.2291,  0.5524,  0.1327,
         -0.2803, -0.3484,  0.1801,  0.4324]], grad_fn=<AddmmBackward0>)


After ReLU : tensor([[0.0000, 0.0896, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0387, 0.2980, 0.0000, 0.0000, 0.0761, 0.3098, 0.0652, 0.0000, 0.0000,
         0.0000, 0.4092],
        [0.0000, 0.0000, 0.0308, 0.0000, 0.0000, 0.0000, 0.0000, 0.3762, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.1843, 0.

In [14]:
# Sequential 모델 사용해보기
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.1424, -0.1764,  0.1552,  0.2257, -0.0880, -0.0195, -0.2426,  0.1256,
         -0.1192, -0.2020],
        [ 0.1301, -0.1423,  0.1974,  0.1977, -0.0337,  0.0075, -0.1553,  0.2390,
          0.0063, -0.1460],
        [ 0.1526, -0.2019,  0.2311,  0.1861, -0.0467, -0.0070, -0.2312,  0.2610,
         -0.0620, -0.1109]], grad_fn=<AddmmBackward0>)


In [16]:
# Softmax 활성화 함수 사용해보기
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1161, 0.0844, 0.1176, 0.1262, 0.0922, 0.0987, 0.0790, 0.1142, 0.0894,
         0.0823],
        [0.1094, 0.0833, 0.1170, 0.1170, 0.0928, 0.0967, 0.0822, 0.1219, 0.0966,
         0.0830],
        [0.1129, 0.0792, 0.1221, 0.1167, 0.0925, 0.0962, 0.0769, 0.1258, 0.0911,
         0.0867]], grad_fn=<SoftmaxBackward0>)


In [ ]:
# 모델의 각 계층과 매개변수 출력
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer : {name} | Size : {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[ 0.0084,  0.0027,  0.0087,  ...,  0.0142,  0.0268, -0.0202],
        [ 0.0298,  0.0034,  0.0176,  ...,  0.0019, -0.0134, -0.0257]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([0.0219, 0.0352], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[ 0.0050, -0.0088,  0.0330,  ..., -0.0171, -0.0203, -0.0199],
        [ 0.0393, -0.0320, -0.0013,  ..., -0.0358,  0.0203, -0.0434]],
       device='cuda:0', grad_fn